## Part B: Creating a data frame with Latitude and Longitude.


In [ ]:
%pip install beautifulsoup4 # Installing the BeautifulSoup4
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup
import requests # library to handle requests

In [ ]:
# Installing the Conda
!conda update -n base -c defaults conda --yes

In [ ]:
%pip install requests
import requests
# Loading data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# Using beautiful soup to parse the HTML/XML codes.
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_wikipedia_page,'html')
# print(soup.prettify())

In [ ]:
My_table = soup.find('table', {"class": 'wikitable sortable'})

In [ ]:
New_Table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# Extracting a clean form of the table
for tr_cell in New_Table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [ ]:
Unique_Postal = set(Postcode)
print('num of unique Postal codes:', len(Unique_Postal))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in Unique_Postal:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

In [ ]:
%pip install geocoder

In [ ]:
import geocoder

In [ ]:
Table_PostalCode = soup.find('table')
fields = Table_PostalCode.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_PostalCode = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_PostalCode.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_PostalCode.head()

In [ ]:
df_PostalCode['Borough'].replace('Not assigned', np.nan, inplace=True)
df_PostalCode.dropna(subset=['Borough'], inplace=True)

df_PostalCode.head()

In [ ]:
df_PostalCodeNeighbor = df_PostalCode.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_PostalCodeNeighbor.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_PostalCodeNeighbor

In [ ]:
df_PostalCodeNeighbor['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)
df_PostalCodeNeighbor

In [ ]:
df_PostalCodeNeighbor.shape

In [ ]:
df_geospatial_data = pd.read_csv('http://cocl.us/Geospatial_data')
df_geospatial_data.columns = ['Postcode', 'Latitude', 'Longitude']
df_geospatial_data.head()

In [ ]:
df_Post = pd.merge(df_PostalCodeNeighbor, df_geospatial_data, on=['Postcode'], how='inner')
df_Full_Table = df_Post[[ 'Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude',]].copy()
df_Full_Table.head()